In [ ]:
from sklearn.model_selection import KFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow as tf2
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time, math
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

tf.set_random_seed(777)  

xy_data = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)
print(xy_data.shape)
xy_te_data = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)
print(xy_te_data.shape)

np.random.shuffle(xy_data)
np.random.shuffle(xy_te_data)

xy_data=xy_data.reshape(-1,21)
xy_te_data=xy_te_data.reshape(-1,21)


y_class=(xy_data[:,-1].reshape(-1,1))
enc = OneHotEncoder(categories='auto')
enc.fit(y_class)
y_class_onehot = enc.transform(y_class).toarray()

y_class_te=(xy_te_data[:,-1].reshape(-1,1))
enc_te = OneHotEncoder(categories='auto')
enc_te.fit(y_class_te)
y_class_onehot_te = enc_te.transform(y_class_te).toarray()

x_data_training = xy_data[:, :-5]
print(x_data_training.shape)
y_data_training = y_class_onehot[:, :]
print(y_data_training.shape)
x_data_test = xy_te_data[:, :-5]
print(x_data_test.shape)
y_data_test = y_class_onehot_te[:, :]
print(y_data_test.shape)

tf.reset_default_graph()
nb_classes=3
# nb_classes=10
# nb_classes=9

X = tf.placeholder(tf.float32, shape=[None, 16]) #X_16
Y = tf.placeholder(tf.float32, shape=[None, nb_classes])

W1 = tf.get_variable("weight1", shape=[16, 64], initializer=tf2.initializers.GlorotUniform())
W2 = tf.get_variable("weight2", shape=[64, 128], initializer=tf2.initializers.GlorotUniform())
W3 = tf.get_variable("weight3", shape=[128, 64], initializer=tf2.initializers.GlorotUniform())
W4 = tf.get_variable("weight4", shape=[64, nb_classes], initializer=tf2.initializers.GlorotUniform())


b1 = tf.Variable(tf.random_normal([64]), name='bias1')
b2 = tf.Variable(tf.random_normal([128]), name='bias2')
b3 = tf.Variable(tf.random_normal([64]), name='bias3')
b4 = tf.Variable(tf.random_normal([nb_classes]), name='bias4')


L2 = tf.nn.relu(tf.matmul(X, W1) + b1)
L3 = tf.nn.relu(tf.matmul(L2, W2) + b2)
L4 = tf.nn.relu(tf.matmul(L3, W3) + b3)
logits = tf.matmul(L4, W4) + b4

hypothesis= tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train_op  = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
prediction = tf.argmax(hypothesis, 1)
label = tf.argmax(Y, 1)
prediction_corrected = tf.equal(label, prediction)
accuracy = tf.reduce_mean(tf.cast(prediction_corrected, tf.float32))   

from sklearn.metrics import r2_score

init = tf.global_variables_initializer()

def run_train(sess, train_x, train_y):
    print("\nStart training")
    sess.run(init)
    batch_size = 50
    for epoch in range(100):
        total_batch = int(train_x.shape[0] / batch_size)
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_y = train_y[i*batch_size:(i+1)*batch_size]
            _, c, acc = sess.run([train_op, cost, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Epoch #%d step=%d cost=%f accuracy=%f" % (epoch, i, c, acc))

def cross_validate(sess, split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    for train_idx, val_idx in kf.split(x_data_training, y_data_training):
        train_x = x_data_training[train_idx]
        train_y = y_data_training[train_idx]
        val_x = x_data_training[val_idx]
        val_y = y_data_training[val_idx]
        run_train(sess, train_x, train_y)
        results.append(sess.run(accuracy, feed_dict={X: val_x, Y: val_y}))
    return results
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = cross_validate(sess)
    print("Cross-validation accuracy: %s" % result)
    print("Test cost: %f" % sess.run(cost, feed_dict={X: x_data_test, Y: y_data_test}))
    print("Test accuracy: %f" % sess.run(accuracy, feed_dict={X: x_data_test, Y: y_data_test}))
    y_predict_tr = sess.run(prediction, feed_dict={X: x_data_training})
    y_predict = sess.run(prediction, feed_dict={X: x_data_test})
    y_label = sess.run(label, feed_dict={X: x_data_test, Y: y_data_test})
    saver = tf.train.Saver()
    saver.save(sess, 'save_path')
    print(y_predict)
    print(y_label)
    np.savetxt("save_path" , y_predict, delimiter=',',fmt='%4f')
    np.savetxt("save_path" ,y_label, delimiter=',',fmt='%4f')